In [ ]:
#functions
import pandas as pd
import math

#AGB(coniferous tree)
def bio_C(D):
    bio = 1.031 * math.exp(-1.51+1.157*math.log(D)+0.518*(math.log(D))**2-0.067*(math.log(D))**3)
    return bio

#AGB(deciduous broadleaf tree)
def bio_DA(D):
    bio = 1.031 * math.exp(-1.501+1.375*math.log(D)+0.464*(math.log(D))**2-0.061*(math.log(D))**3)
    return bio

#AGB(evergreen broadleaf tree)
def bio_EA(D):
    bio = 1.031 * math.exp(-1.698+1.851*math.log(D)+0.239*(math.log(D))**2-0.031*(math.log(D))**3)
    return bio    

#AGB(density known)
def bio_dense(D,dense):
    bio = 1.029 * math.exp(-1.196+1.622*math.log(D)+0.338*(math.log(D))**2-0.044*(math.log(D))**3+0.708*math.log(dense))
    return bio


#measuring biomass. measure == "B" if you want to measure biomass. measure == "C" if you want to measure carbon stock.（default is "C".）
#x is a pandas series that has "DBH","Wood density (g/cm3)","type" keys.
def measurebio(x,measure="C",ignore=False):
    if x["Wood density (g/cm3)"] ==2:
        biomass = bio_C(x["DBH"])/1000
    elif x["Wood density (g/cm3)"] == 3:
        biomass = bio_DA(x["DBH"])/1000
    elif x["Wood density (g/cm3)"] == 4:
        biomass = bio_EA(x["DBH"])/1000
    elif x["Wood density (g/cm3)"] < 2:
        biomass = bio_dense(x["DBH"],x["Wood density (g/cm3)"])/1000
    else:
        if not ignore:
            raise parameter_Error("Wood density is invalid")

    if measure == "C":
        if x["type"] == 'EB' or x["type"]=='DB':
            biomass = biomass * 0.48
        elif x["type"]=='EC':
            biomass = biomass * 0.51
        else:
            raise parameter_Error("You must specify type as EB,DB or EC")
    else:
        if not ignore:
            raise parameter_Error("measure parameter is invalid")
    return biomass

#dominant species
def max_species(frame,co):
    data = frame.sort_values(co,ascending=False)
    top1 = data.iloc[0]["species"]
    if len(data) == 2:
        top2=data.iloc[1]["species"]
        top3=None
    elif len(data)==1:
        top2=top3=None
    else:
        top2=data.iloc[1]["species"]
        top3=data.iloc[2]["species"]
    return top1,top2,top3


In [ ]:
#aggregate dominant species, biomass and taxonomic diversity (make filelist)

import pandas as pd
import math

#df[["species","type","region"]]=df[["species","type","region"]].astype("string")
class parameter_Error(Exception):
    pass


#read inventory data
df=pd.read_csv("./inventory_current.csv",index_col=0)

flist = df.groupby("ID")["forest_type"].unique().reset_index().set_axis(["ID","forest_type"],axis=1)

df["bio"] = df[["DBH","Wood density (g/cm3)","type"]].apply(measurebio,measure="C",axis=1)
df["bio"]=df[["DBH","bio"]].apply(lambda x: x["bio"]*2.5 if x["DBH"]<18 else x["bio"], axis=1)

a = df.groupby(["ID","species"])['bio'].agg([("biomass","sum"),("counts_s","count")]).reset_index()

type_df = df.groupby(["ID","type"])['bio'].sum().unstack().reset_index()
type_df[["EB","DB","EC"]]=type_df[["EB","DB","EC"]].fillna(0)
type_df["allC"] = type_df["EB"]+type_df["DB"]+type_df["EC"]
y=df.groupby(["ID"])["DBH"].agg([("min_DBH","min"),("max_DBH","max"),("counts","count")])
z=df.groupby(["ID"])["species"].nunique()

type_df=pd.merge(type_df,y,on="ID")
type_df=pd.merge(type_df,z,on="ID")

b = a.groupby("ID").apply(max_species,co = "counts_s").reset_index().set_axis(["ID","dominant_c"],axis=1)
c = b["dominant_c"].apply(pd.Series).set_axis([f"dominant_c{i}" for i in range(1,4)],axis=1)
c["ID"]=b["ID"]

d = a.groupby("ID").apply(max_species,co = "biomass").reset_index().set_axis(["ID","dominant_b"],axis=1)
e = d["dominant_b"].apply(pd.Series).set_axis([f"dominant_b{i}" for i in range(1,4)],axis=1)
e["ID"]=d["ID"]

b=pd.merge(b,c,on="ID").drop("dominant_c",axis=1)
b=pd.merge(b,e,on="ID")

merged=pd.merge(flist,b,on="ID")
merged=pd.merge(merged,type_df,on="ID")

#area of current plots is 0.1ha.
#merged["area"]=0.1
merged["EB"]=merged["EB"]/merged["area"]
merged["DB"]=merged["DB"]/merged["area"]
merged["EC"]=merged["EC"]/merged["area"]
merged["allC"]=merged["allC"]/merged["area"]

#merged is filelist of inventory data
#in historical data, K057, S158 and S180 are removed because their minimum DBH calss are not 10cm.


In [ ]:
#make dominant species table (TableS1)
import pandas as pd
import numpy as np
import math

Past = pd.read_csv("./inventory_historical.csv")
present = pd.read_csv("./inventory_current.csv")
Natural = present[present["forest_type"]=="natural"]
Df = pd.read_csv("./filelist_historical.csv")

past,natural,df = Past.copy(),Natural.copy(),Df.copy()

#select region (shikoku or kyushu)
region="kyushu"

df = df[df["region"]== region]
past = past[past["region"]== region]
natural = natural[natural["region"]== region]

natural = natural[["ID","species","DBH"]]
natural_area = round(natural["ID"].nunique()*0.1,2)

#measure basal area (m2/ha) and density (individuals/ha)
past["BA"] = (past["DBH"]/2)**2*math.pi/10000
natural["BA"] =  natural.apply(lambda x :((x["DBH"]/2)**2*math.pi)/10000 if x["DBH"]>=18 else ((x["DBH"]/2)**2*math.pi)/4000,axis=1)

natural["count"] = natural.apply(lambda x : 2.5 if x["DBH"]<=18 else 1,axis=1)

natural = natural.groupby("species")[["BA","count"]].sum()
past = past.groupby("species")["BA"].agg([("BA","sum"),("count","count")])

past[["BA","density"]],natural[["BA","density"]] =past[["BA","count"]]/df["area"].sum(),natural[["BA","count"]]/natural_area

#measuring total
pba,nba = past["BA"].sum(),natural["BA"].sum()
pdn,ndn = past["density"].sum(),natural["density"].sum()

#extract dominant species only
past.sort_values("BA",ascending=False,inplace=True)
natural.sort_values("BA",ascending=False,inplace=True)
pastdom,naturaldom = past.iloc[0:10,0],natural.iloc[0:10,0]
lists = pastdom.index.tolist()+naturaldom.index.tolist()
natural=natural[natural.index.isin(lists)]
past=past[past.index.isin(lists)]


otpba,otnba = pba-past["BA"].sum(), nba-natural["BA"].sum()
otpdn,otndn = pdn-past["density"].sum(),ndn-natural["density"].sum()

#calculate percentage and make dataframe
past[["BA_%","density_%"]],natural[["BA_%","density_%"]] = round(past[["BA","density"]]*100/[pba,pdn],2),round(natural[["BA","density"]]*100/[nba,ndn],2)
past["BA"],natural["BA"] = round(past["BA"],2),round(natural["BA"],2)
past["density"],natural["density"] =round(past["density"],2),round(natural["density"],2)

past=past[["BA","BA_%","density","density_%"]]
natural=natural[["BA","BA_%","density","density_%"]]

Ptype,Ntype = Past.drop_duplicates("species")[["species","type"]],Natural.drop_duplicates("species")[["species","type"]]

totype = pd.concat([Ptype,Ntype],axis=0).reset_index().drop_duplicates("species").drop(columns=["index"])

past,natural = pd.merge(past,totype,on="species",how="left"),pd.merge(natural,totype,on="species",how="left")
past.set_index(["type","species"],inplace=True)
natural.set_index(["type","species"],inplace=True)

past.sort_index(level=0,inplace=True,ascending=False)
natural.sort_index(level=0,inplace=True,ascending=False)

#sort BA by type
past = past.groupby("type", group_keys=False).apply(lambda x: x.sort_values("BA", ascending=False))
natural = natural.groupby("type", group_keys=False).apply(lambda x: x.sort_values("BA", ascending=False))

pastall=pd.DataFrame(columns=["BA","BA_%","density","density_%"],index=[["",""],["others","total"]],data=[[round(otpba,2),round(otpba*100/pba,2),round(otpdn,2),round(otpdn*100/pdn,2)],[round(pba,2),100.0,round(pdn,2),100.0]])
naturalall=pd.DataFrame(columns=["BA","BA_%","density","density_%"],index=[["",""],["others","total"]],data=[[round(otnba,2),round(otnba*100/nba,2),round(otndn,2),round(otndn*100/ndn,2)],[round(nba,2),100.0,round(ndn,2),100.0]])

past=pd.concat([past,pastall])
past=past.reset_index()
natural=pd.concat([natural,naturalall])
natural=natural.reset_index()

dom = pd.merge(past,natural,on=["level_0","level_1"],how="outer",suffixes=("_past","_current"),)

In [3]:
#make NMDS dataframe (Fig2)
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

Past = pd.read_csv("./inventory_historical.csv")
past = Past.copy()
Curr = pd.read_csv("./inventory_current.csv")
curr = Curr[Curr["forest_type"]=="natural"]

#select region (shikoku or kyushu)
region="kyushu"

past = past[past["region"]==region]
curr = curr[curr["region"]==region]

past["BA"] = ((past["DBH"]/2)**2*math.pi)/10000
curr["BA"] = curr.apply(lambda x : ((x["DBH"]/2)**2)/10000 if x["DBH"]>=18 else ((x["DBH"]/2)**2)/4000,axis=1)

pdom = past.groupby("species")["BA"].sum().sort_values(ascending=False)/past["BA"].sum()*100
cdom = curr.groupby("species")["BA"].sum().sort_values(ascending=False)/curr["BA"].sum()*100

#extract dominant species (relative BA >= 1%)
psp=pdom[pdom>=1]
csp=cdom[cdom>=1]

pastba=past.groupby(["ID","species"])["BA"].sum().reset_index()
pastba=pastba.pivot(index="ID",columns="species",values="BA").fillna(0)
pastba=pastba[list(psp.index)]
pastba=pd.merge(pd.DataFrame(index=pastba.index,columns=["group"],data=["past"]*len(pastba)),pastba,left_index=True,right_index=True)

currba=curr.groupby(["ID","species"])["BA"].sum().reset_index()
currba=currba.pivot(index="ID",columns="species",values="BA").fillna(0)
currba=currba[list(csp.index)]
currba["group"]="current"

nmds = pd.concat([pastba,currba],axis=0).fillna(0)

#nmds is dataframe for NMDS analysis
#NMDS analysis is performed in R.

In [ ]:
#FigS2 violinplot 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyfunc as pf

past = pd.read_csv("./filelist_historical.csv",index_col=0)
curr = pd.read_csv("./filelist_current.csv")

past_k,past_s = past[past["region"]=="kyushu"], past[past["region"]=="shikoku"]
curr_k,curr_s = curr[curr["region"]=="kyushu"], curr[curr["region"]=="shikoku"]

past_k["forest_type"],past_s["forest_type"]="past","past"

#ds and dk are also used GLM
ds,dk=pd.concat([past_s, curr_s],axis=0), pd.concat([past_k,curr_k],axis=0)
d= pd.concat([ds,dk],axis=0)

sns.set_palette("tab10")

fig,ax = plt.subplots(1,2,figsize=(12,6))

dk.reset_index(drop=True,inplace=True)
ds.reset_index(drop=True,inplace=True)
ds.iloc[293,:],ds.iloc[296,:]=ds.iloc[296,:],ds.iloc[293,:]
sns.violinplot(y="allC",x="forest_type",data=dk,ax=ax[1],density_norm="area",bw_adjust=0.7,color="#bbbbbb")
sns.violinplot(y="allC",x="forest_type",data=ds,ax=ax[0],density_norm="area",bw_adjust=0.7,color="#bbbbbb")
ax[0].set_ylim(0,370)
ax[1].set_ylim(0,370)
ax[0].set_xlabel("")
ax[1].set_xlabel("")
ax[0].set_xticklabels(["past","natural","artificial"],fontsize=20)
ax[1].set_xticklabels(["past","natural","artificial"],fontsize=20)
ax[0].set_yticklabels([0,50,100,150,200,250,300,350],fontsize=16)
ax[1].set_yticklabels([0,50,100,150,200,250,300,350],fontsize=16)
ax[0].set_ylabel("biomass (Mg C/ha)",fontsize=20)
ax[1].set_ylabel("")
ax[0].set_title("Shikoku",fontsize=20)
ax[1].set_title("Kyushu",fontsize=20)

In [ ]:
#Fig1 and FigS4 DBH histgram

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Past = pd.read_csv("./inventory_historical.csv")
File= pd.read_csv("./filelist_historical.csv")
Curr = pd.read_csv("./inventory_current.csv")

past=Past.copy()
file=File.copy()
curr=Curr.copy()

curr=curr[curr["forest_type"]=="natural"]

#select region (shikoku or kyushu)
region="shikoku"

past = past[past["region"]==region]
curr = curr[curr["region"]==region]
file = file[file["region"]==region]

c_area = 0.1 * curr["ID"].nunique()
h_area = file["area"].sum()

#FigS4a
#past=past[past["species"].isin(["Distylium racemosum", "Quercus salicina"])]
#curr=curr[curr["species"].isin(["Distylium racemosum", "Quercus salicina"])]

#FigS4b 
#past=past[past["species"].isin(["Quercus serrata", "Quercus mongolica", "Cerasus jamasakura"])]
#curr=curr[curr["species"].isin(["Quercus serrata", "Quercus mongolica", "Cerasus jamasakura"])]

c_EC,c_DB,c_EB = curr[curr["type"]=="EC"],curr[curr["type"]=="DB"],curr[curr["type"]=="EB"]
h_EC,h_DB,h_EB = past[past["type"]=="EC"],past[past["type"]=="DB"],past[past["type"]=="EB"]

c_list = [c_EC, c_DB, c_EB]
h_list = [h_EC, h_DB, h_EB]

for i,x in enumerate(c_list):
    c_list[i]=x[["DBH"]].dropna().values.flatten()
for i,x in enumerate(h_list):
    h_list[i]=x[["DBH"]].dropna().values.flatten()


bins=[2.5,7.5,12.5,17.5,18,22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5,92.5,97.5,102.5,107.5,112.5,117.5,122.5,127.5,132.5,137.5,142.5,147.5,152.5,157.5,162.5,167.5,172.5,177.5,182.5,187.5,192.5,197.5,202.5]

fig,axes = plt.subplots(1,2,figsize=(16,8))

counts_c, bins_c, _ = axes[0].hist([c_list[0],c_list[1],c_list[2]], stacked=True, range=[2.5, 202.5], bins=bins, alpha=0.7, label=["EC","DB","EB"],log=True)
counts_h, bins_h, _ = axes[1].hist([h_list[0],h_list[1],h_list[2]], stacked=True, range=[2.5, 202.5], bins=40, alpha=0.7, label=["EC","DB","EB"],log=True)

counts_c[:,[1,2,3]]=counts_c[:,[1,2,3]]*2.5
counts_c[:,3]=counts_c[:,3]+counts_c[:,4]
counts_c=np.delete(counts_c,4,1)

counts_h_scaled = counts_h / h_area
counts_c_scaled = counts_c / c_area
counts_h_scaled= pf.log_ratio(counts_h_scaled,0.002,stack=True)
counts_c_scaled= pf.log_ratio(counts_c_scaled,0.002,stack=True)

plt.style.use("default")
sns.set()
sns.set_style("whitegrid")
sns.set_palette("pastel")
fig,axes = plt.subplots(2,1,figsize=(6,12))

datalist=[counts_h_scaled,counts_c_scaled]
binlist=[bins_h,bins_h]
name=["EC","DB","EB"]

for j,x in enumerate(datalist):
    for i in range(len(x)):
        axes[j].bar(binlist[j][:-1] ,x.iloc[i],align="edge",width=np.diff(binlist[j]) ,bottom=x.iloc[:i].sum(),label=name[i],log=True)

handles, labels = axes[0].get_legend_handles_labels()
ylab=axes[0].get_yticklabels()
axes[0].set_ylabel("density (/ha ,log scale)",fontsize=11)
axes[0].set_xticklabels(np.arange(0,225,25),fontsize=12)
axes[0].set_yticklabels(ylab,fontsize=12)
axes[0].set_ylim(0.002,2000)
axes[0].set_xlim(0,202.5)
#axes[0].set_title("Historical",fontsize=25)

axes[1].set_xlabel("DBH (cm)",fontsize=20)
axes[1].set_ylabel("density (/ha ,log scale)",fontsize=11)
axes[1].set_xticklabels(np.arange(0,225,25),fontsize=12)
axes[1].set_yticklabels(ylab,fontsize=12)
axes[1].set_ylim(0.002,2000)
axes[1].set_xlim(0,202.5)
#axes[1].set_title("Current",fontsize=25)


In [ ]:
#analyzing density of large-diameter trees by zero_inflated negative binomial GLM
import pandas as pd
import numpy as np 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels as stm
import seaborn as sns

Past = pd.read_csv("./inventory_historical.csv")
pFile = pd.read_csv("./filelist_historical.csv")
Curr = pd.read_csv("./inventory_current.csv")
cFile = pd.read_csv("./filelist_current.csv")

past=Past.copy()
pfile=pFile.copy()
curr=Curr.copy()
cfile = cFile.copy()

curr = curr[curr["forest_type"]=="natural"]
cfile = cfile[cfile["forest_type"]=="natural"]

#select region (shikoku or kyushu)
region="kyushu"

past = past[past["region"]==region]
curr = curr[curr["region"]==region]
pfile = pfile[pfile["region"]==region]
cfile = cfile[cfile["region"]==region]

past = past[past["DBH"]>=50]
curr = curr[curr["DBH"]>=50]

h_large = past.groupby("ID")["DBH"].count()
h_large = pd.merge(h_large,pfile,on="ID",how="outer")
h_large["dens"] = h_large["DBH"]/h_large["area"]
h_large["dens"].fillna(0,inplace=True)
h_large["historical"] = 1
h_large = h_large[["ID", "DBH", "dens", "historical"]]

c_large = pd.DataFrame(curr.groupby("ID")["DBH"].count())
c_large = pd.merge(c_large,cfile,on="ID",how="outer")
c_large["dens"] = c_large["DBH"]/0.1
c_large["dens"].fillna(0,inplace=True)
c_large["historical"] = 0

larges = pd.concat([h_large,c_large],axis=0)

h_ave, c_ave = larges[larges["historical"]==1]["dens"].mean(), larges[larges["historical"]==0]["dens"].mean()

model=sm.ZeroInflatedNegativeBinomialP(endog=np.asarray(larges["dens"]),exog=np.asarray(larges["historical"]))
result = model.fit(disp=0)
print(result.summary())

In [ ]:
#GLM(AGB)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns

#use ds or dk (made in "violin plot" cell)
data = ds #dk

data["historical"] = data.apply(lambda x : 1 if x["forest_type"]=="past" else 0,axis=1)
data["natural"] = data.apply(lambda x : 1 if x["forest_type"]=="natural" else 0,axis=1)

formula = "allC ~ historical + natural"
link = sm.families.links.log()
model = smf.glm(formula, data=data , family=sm.families.Gamma(link=link))
result = model.fit()

print(result.summary())

In [ ]:
#Fig3 taxonomic diversity (prepare iNEXT input file)
import pandas as pd
import numpy as np

Past = pd.read_csv("./inventory_historical.csv")
Curr = pd.read_csv("./inventory_current.csv")

past = Past.copy()
curr = Curr[Curr["forest_type"]=="natural"]

#select region (shikoku or kyushu)
region="kyushu"

past = past[past["region"]==region]
curr = curr[curr["region"]==region]

curr["count"] = curr.apply(lambda x : 2.5 if x["DBH"]<=18 else 1,axis=1)

#make iNEXT input file
r_past,r_curr = past.groupby(["ID","species"])["DBH"].count().reset_index() ,curr.groupby(["ID","species"])["count"].count().reset_index()
r_curr["count"] = np.floor(r_curr["count"])
r_past,r_curr = r_past.pivot(index="species",columns="ID",values="DBH"), r_curr.pivot(index="species",columns="ID",values="count")

rare=pd.concat([r_past,r_curr],axis=1).fillna(0)

In [ ]:
#Fig3 FigS3 taxonomic diversity　(plot and GLM)
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

#read iNEXT output file (shikoku: inext_sresult.csv, kyushu: inext_kresult.csv)
Data = pd.read_csv("./inext_kresult.csv")
File = pd.read_csv("./filelist_historical.csv")

data = Data.copy()
file = File.copy()
file["current"] = 0

#coverage: shikoku = 0.857, kyushu = 0.78
datas = pd.merge(data,file,left_on = "Cmax = 0.78",right_on="ID",how="left")
datas["area"].fillna(0.1,inplace=True)
datas["current"].fillna(1,inplace=True)
datas["current_a"] = datas["current"].replace({1:"current",0:"historical"})
datas.rename(columns={"q = 0":"diversity"},inplace=True)

#make model based on area-diversity relationship (GLM)
data=datas[datas["current"]==0]
data["logarea"] = np.log(data["area"])
formula = "diversity ~ logarea"
link = sm.families.links.Log()
model = smf.glm(formula, data=data , family=sm.families.Gamma(link=link))
result = model.fit()
print(result.summary())

#t_test
from scipy import stats
historical = datas[datas["current"]==0]["diversity"]
current = datas[datas["current"]==1]["diversity"]
t_stat, p_value = stats.ttest_ind(historical, current, equal_var=False)
print(f"T-statistic: {t_stat}, P-value: {p_value}")

#plot Fig3
fig, ax = plt.subplots(1,2,figsize=(13.5,4.6))

sns.boxplot(x="current", y="diversity", data=datas, palette=['#ffb482', '#a1c9f4'],ax=ax[0])

ax[0].set_ylim(0,41)
ax[0].set_ylabel("Taxonomic diversity",fontsize=15)
ax[0].set_xticklabels(["historical","current"],fontsize=15)
ax[0].set_xlabel("")
ax[0].set_yticks([0,10,20,30,40])
ax[0].set_yticklabels([0,10,20,30,40],fontsize=15)
plt.grid(False)


#FigS3 scatter plot and model line
#shikoku: 0.001-4.3, kyushu: 0.001-1.8
x=pd.DataFrame(np.arange(0.001,1.8,0.001),columns=["area"])
sns.scatterplot(x="area",y="diversity", data=datas, hue="current_a", palette=['#ff7f0e', '#1f77b4'],ax=ax[1])

x["logarea"] = np.log(x["area"])
x['y_pred'] = result.predict(x["logarea"])

data.sort_values("logarea",inplace=True)

handles, labels = ax[1].get_legend_handles_labels()
ax[1].plot(x["area"],x["y_pred"],color="red",linewidth=2)

ax[1].set_ylim(0,41)
ax[1].set_ylabel("Taxonomic diversity",fontsize=15)
ax[1].set_xlabel("area",fontsize=15)
ax[1].set_yticks([0,10,20,30,40])
ax[1].set_yticklabels([0,10,20,30,40],fontsize=15)
ax[1].legend(title="",handles=handles ,fontsize=12,labels=["historical","current"])

#shikoku ver.
#ax[1].set_xticks([0,1,2,3,4])
#ax[1].set_xticklabels(["0","1.0","2.0","3.0","4.0"],fontsize=15)
#ax[1].text(0,35,"Coverage=0.857",fontsize=15)

#kyushu ver.
ax[1].text(0,35,"Coverage=0.780",fontsize=15)
ax[1].set_xticks([0,0.5,1,1.5,2])
ax[1].set_xticklabels(["0","0.5","1.0","1.5","2.0"],fontsize=15)

#fig.savefig("./kyushu.png",dpi=300,bbox_inches="tight")

In [ ]:
#historical change of human impact and its effect on tree density (Fig4)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu

Past = pd.read_csv("./inventory_historical.csv")
pFile = pd.read_csv("./filelist_historical.csv")
Curr = pd.read_csv("./inventory_current.csv")
cFile = pd.read_csv("./filelist_current.csv")

past=Past.copy()
pfile=pFile.copy()
curr=Curr.copy()
cfile = cFile.copy()

curr = curr[curr["forest_type"]=="natural"]
cfile = cfile[cfile["forest_type"]=="natural"]

#select region (shikoku or kyushu)
region="kyushu"

past = past[past["region"]==region]
curr = curr[curr["region"]==region]
pfile = pfile[pfile["region"]==region]
cfile = cfile[cfile["region"]==region]

past = past[past["DBH"]>=50]
curr = curr[curr["DBH"]>=50]

h_large = past.groupby("ID")["DBH"].count()
h_large = pd.merge(h_large,pfile,on="ID",how="outer")
h_large["dens"] = h_large["DBH"]/h_large["area"]
h_large["dens"].fillna(0,inplace=True)

c_large = pd.DataFrame(curr.groupby("ID")["DBH"].count())
c_large = pd.merge(c_large,cfile,on="ID",how="outer")
c_large["dens"] = c_large["DBH"]/0.1
c_large["dens"].fillna(0,inplace=True)

#distance from human structures (road, building) is calculated by QGIS
h_large = pd.merge(h_large,hhuman,on="ID",how="inner")
c_large = pd.merge(c_large,chuman,on="ID",how="inner")

h_large = h_large[h_large["radius"]<=1000]

#add 1 if distance is 0
h_large["log1900_dist"]= np.log(h_large["1900_dist"].replace(0,1))
h_large["log1950_dist"]= np.log(h_large["1950_dist"].replace(0,1))
c_large["logdist"]= np.log(c_large["distance"].replace(0,1))
h_large["logdens"] = np.log(c_large["dens"].replace(0,1))  
c_large["logdens"] = np.log(c_large["dens"].replace(0,1))  

model = smf.glm(formula="big ~ log1900_dist", data=h_large, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(h_large["area"]))
model2 = smf.glm(formula="big ~ log1950_dist", data=h_large, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(h_large["area"]))
model3 = smf.glm(formula="big ~ logdist", data=c_large, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(0.1))

result = model.fit()
result2 = model2.fit()
result3 = model3.fit()

print(result.summary())
print(result2.summary())
print(result3.summary())